In [ ]:
import pandas as pd
import torch

from loki_datasets_features import LokiDataModule


In [ ]:
dm = LokiDataModule(batch_size=1024)

In [ ]:
from loki_datasets_features import LokiTrainValDataset, LokiTestDataset
ab = LokiTrainValDataset()
a_encoder_train = ab.label_encoder
a_numeric_columns = ab.numeric_columns
a_imputer_num = ab.imputer_num
cd = LokiTestDataset(label_encoder=a_encoder_train, numeric_columns=a_numeric_columns, imputer_num=a_imputer_num)
a =set(ab.numeric_columns)
b =set(cd.numeric_columns)
print(len(a),len(b))
print(a)

In [ ]:
train_dataiter =iter(dm.train_dataloader())
train_features_list=[]
train_label_list=[]
train_counter = 0
while True:
    img, feature, label = next(train_dataiter)
    train_features_list.append(feature)
    train_label_list.append(label)
    print(train_counter, img.shape, feature.shape, label.shape)
    train_counter +=1

In [ ]:
val_dataiter =iter(dm.val_dataloader())
val_features_list=[]
val_label_list=[]
val_counter = 0
while True:
    img, feature, label = next(val_dataiter)
    val_features_list.append(feature)
    val_label_list.append(label)
    print(val_counter, img.shape, feature.shape, label.shape)
    val_counter +=1

In [ ]:
test_dataiter = iter(dm.test_dataloader())
test_features_list = []
test_label_list = []
test_counter = 0
while True:
    img, feature, label = next(test_dataiter)
    test_features_list.append(feature)
    test_label_list.append(label)
    print(test_counter, img.shape, feature.shape, label.shape)
    test_counter += 1

In [ ]:
import torch
train_features = torch.concat(train_features_list, dim=0).numpy()
train_labels = torch.concat(train_label_list, dim=0).numpy()
val_features = torch.concat(val_features_list, dim=0).numpy()
val_labels = torch.concat(val_label_list, dim=0).numpy()
test_features = torch.concat(test_features_list, dim=0).numpy()
test_labels = torch.concat(test_label_list, dim=0).numpy()

In [ ]:
# Sklear
X_train = pd.DataFrame(train_features)
y_train = pd.DataFrame(train_labels.ravel())
X_val = pd.DataFrame(val_features)
y_val = pd.DataFrame(val_labels.ravel())
X_test = pd.DataFrame(test_features)
y_test = pd.DataFrame(test_labels.ravel())

In [ ]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

In [ ]:
import numpy as np
from time import time
from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_selector as selector
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

np.random.seed(42)
numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="mean")),
        ("scaler", StandardScaler())
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, selector(dtype_include="number")),
    ]
)


clf = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", RandomForestClassifier(max_depth=20, n_estimators=50, max_features=25))]
)

t0 = time()
clf.fit(X_train, y_train)
print("fit done in %0.3fs" % (time() - t0))
y_my_test = clf.predict(X_test)
y_my_val = clf.predict(X_val)
print("model score on validation data: %.3f" % clf.score(X_val, y_val))
print("model score on test data: %.3f" % clf.score(X_test, y_test))

In [ ]:
from time import time
from matplotlib import pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report

rf_class_report = pd.DataFrame(classification_report(y_true = y_test, y_pred = y_my_test, output_dict=True)).transpose()
rf_class_report.to_csv(f"paper/tables/RF_03032023_sklearn_class_report.csv", sep=";")
print("Predicting people's names on the test set")
t0 = time()
y_pred = clf.predict(X_test)
print("done in %0.3fs" % (time() - t0))

#target_names = list(y_test)
all_cls  =list(np.unique(a_encoder_train.inverse_transform(y_test)))
print(classification_report(y_true = y_test, y_pred = y_pred))
disp =ConfusionMatrixDisplay.from_estimator(
    clf, X_test, y_test, xticks_rotation="vertical",cmap="viridis",
display_labels=all_cls)
fig = disp.figure_
fig.set_figwidth(20)
fig.set_figheight(20)
#fig.suptitle('Plot of confusion matrix')
plt.tight_layout()
plt.savefig('paper/figures/random_forestsklearn_confusion_matrix.jpg')
plt.show()